<a href="https://colab.research.google.com/github/RayChen0623/Funnel-Analysis/blob/main/Funnel_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
! pip install pandasql

In [110]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import gspread
import datetime as dt
import pandasql
import os
import plotly
import gspread_dataframe as gd
from oauth2client.client import GoogleCredentials
from google.colab import drive
from google.colab import auth
import plotly

auth.authenticate_user()

In [111]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
ls "/content/drive/My Drive/Funnel Analysis/Dataset"

home_page_table.csv             payment_page_table.csv  user_table.csv
payment_confirmation_table.csv  search_page_table.csv


### Load Data

In [113]:
data_drive = os.listdir('/content/drive/My Drive/Funnel Analysis/Dataset/') 

In [114]:
df_list = []

for x in data_drive:
  df_list.append(pd.read_csv('/content/drive/My Drive/Funnel Analysis/Dataset/' + x, encoding='latin-1')) 

In [115]:
df_home_page_table = df_list[0].rename(columns={"user_id": "user_id", "page": "home_page"})
df_user_table = df_list[1]
df_search_page_table = df_list[2].rename(columns={"user_id": "user_id", "page": "search_page"})
df_payment_confirmation = df_list[3].rename(columns={"user_id": "user_id", "page": "confirmation_page"})
df_payment_page_table = df_list[4].rename(columns={"user_id": "user_id", "page": "payment_page"})

In [116]:
df = df_user_table.merge(df_home_page_table, on = 'user_id', how = "left").merge(
    df_search_page_table, on = 'user_id', how = "left").merge(
        df_payment_page_table, on = 'user_id', how = "left").merge(
            df_payment_confirmation, on = 'user_id', how = "left")

In [117]:
funnel_df = pd.melt(df, id_vars=['user_id', 'date', 'device', 'sex'], value_vars=['home_page', 'search_page', 'payment_page', 'confirmation_page'],
        var_name='page', value_name='val').groupby("page").count()['val'].reset_index().sort_values(by = "val", ascending = False)

In [118]:
funnel_df.columns = ['step', 'val']

In [119]:
funnel_df['pct'] = (100 - 100 * abs(funnel_df['val'].pct_change()).fillna(0)).astype(int)
funnel_df['val-1'] = funnel_df['val'].shift(1)
funnel_df['pct_from_first'] = (funnel_df['val'] / funnel_df['val'].loc[0] * 100).fillna(0).astype(int)

In [120]:
import plotly.graph_objects as go

In [121]:
def plot_stacked_funnel(events, steps, col=None, from_date=None, to_date=None, step_interval=0):
    """
    Function used for producing a (stacked) funnel plot.
    """
    # create list to append each trace to
    # this will be passed to "go.Figure" at the end
    data = []

    
    funnel_df = events
    dict_ = {'Page View': funnel_df}
    title = 'Funnel plot'

    for t in dict_.keys():
        trace = go.Funnel(
            name=t,
            y=dict_[t].step.values,
            x=dict_[t].val.values,
            textinfo="value+percent previous"
        )
        data.append(trace)

    layout = go.Layout(margin={"l": 180, "r": 0, "t": 30, "b": 0, "pad": 0},
                       funnelmode="stack",
                       showlegend=True,
                       hovermode='closest',
                       title='Funnel plot per {}'.format(col),
                       legend=dict(orientation="v",
                                   bgcolor='#E2E2E2',
                                   xanchor='left',
                                   font=dict(
                                       size=12)
                                   )
                       )

    return go.Figure(data, layout)

In [122]:
fig = plot_stacked_funnel(funnel_df, ['home_page', 'search_page', 'payment_page', 'confirmation_page'])

In [123]:
fig

In [124]:
#pandasql.sqldf("SELECT device FROM df_user_table LIMIT 20;", globals())